In [1]:
import itertools
import json
import pickle
import re

import numpy as np
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

from tqdm.notebook import tqdm
tqdm.pandas()

/tmp/ipykernel_4913/1377736200.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open("../../data/pseudowords/CoMaPP_all.json") as json_file:
    data = json.load(json_file)
    
# d+1 since we are not replacing the ke-lex this time!
data = [{"example": d["target1"], "cue": " ".join(d["target1"].split()[:d["query_idx"]+1]), "pseudoword": d["label"]} for d in data]
df = pd.DataFrame.from_dict(data).drop_duplicates(ignore_index=True)
df

,example,cue,pseudoword
0,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
1,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
2,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
3,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
4,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
12128,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
12129,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
12130,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99
12131,"An seiner Hand geht ein Mädchen , einen Kopf k...","An seiner Hand geht ein Mädchen , einen Kopf k...",er99


In [3]:
df['index'] = df['pseudoword'].str.extract('(\d+)').astype(int)
df.set_index('index', inplace=True)

df

,example,cue,pseudoword
index,,,
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99


In [4]:
contextleft = pd.read_pickle("../../data/pseudowords/contextleft_text.pickle")

def update_cue(row):
    output = row[['example', 'cue']]
    if len(row['cue'].split()) == 1:  # if the string in cue is empty (except for the kelex)
        # match the index of row with contextleft['construction_id'] and match contextleft['text'] with row['example'] and create matching_entry
        matching_entry = contextleft.loc[(contextleft['construction_id'] == row.name) & (contextleft['text'] == row['example']), 'contextleft'].tolist()
        if len(matching_entry) > 0:
            # Add the left context to the example and to the cue:
            output = [matching_entry[0] + " " + row['example'], matching_entry[0] + " " + row['cue']]
    return output

# Add the left context if there is no cue up until the pseudoword.
df[["example", "cue"]] = df.apply(update_cue, axis=1)
df

,example,cue,pseudoword
index,,,
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99


In [5]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'construction'}, inplace=True)

result_df = df.groupby(['construction', 'pseudoword']).agg({'example': list, 'cue': list})

result_df

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  
construction pseudoword                                                     
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...  
             erst5       [Trainer Lucien Favre hatte schon seine beiden...  
             gar5        [Es hat Afghanistan nicht stabilisiert und, Es...  
             nicht5      [Es hat, Es hat Afghanistan nicht, Dass es, Da...  
             recht5      [Trainer Lucien Favre hatte schon seine beiden...  
...                                                                    ...  
1884         Gold1884    [Schweigen ist Silber , reden ist Gold, Schwei...  
             Silber1884  [Schweigen ist Silber, Schweigen ist Silber, A...  
             ist1884     [Schweigen ist, Schweigen ist, Angeben ist, Sc...  
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war, Die Vorhut ...  
1987         sehr1987    [Solche Menschen können sich, Solche Menschen ...  

[562 rows x 2 columns]

In [6]:
with open("../../out/definitions.pickle", "rb") as definitions_file:
    definitions = pd.DataFrame.from_dict(pickle.load(definitions_file), orient="index", columns=["definition"])
    
definitions

,definition
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio..."
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de..."
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru..."
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""..."
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ..."
...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g..."
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e..."
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han..."
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z..."


In [7]:
examples = pd.merge(result_df, definitions, how="inner", left_on="construction", right_index=True)
examples

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und, Es...   
             nicht5      [Es hat, Es hat Afghanistan nicht, Dass es, Da...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold, Schwei...   
             Silber1884  [Schweigen ist Silber, Schweigen ist Silber, A...   
             ist1884     [Schweigen ist, Schweigen ist, Angeben ist, Sc...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war, Die Vorhut ...   
1987         sehr1987    [Solche Menschen können sich, Solche Menschen ...   

                                                                definition  
construction pseudoword                                                     
5            Und5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             erst5       Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             gar5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             nicht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             recht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
...                                                                    ...  
1884         Gold1884    Die "Intensivierung_Komparativ:Xist_SilberYist...  
             Silber1884  Die "Intensivierung_Komparativ:Xist_SilberYist...  
             ist1884     Die "Intensivierung_Komparativ:Xist_SilberYist...  
1986         kaum1986    Die Konstruktion "Relativierung:kaumADJ" gehör...  
1987         sehr1987    Die Konstruktion "Intensivierung:sehrV" gehört...  

[562 rows x 3 columns]

##### Generieren neuer Sätze:

Load the vanilla mbart model:

In [10]:
model = MBartForConditionalGeneration.from_pretrained(
    "facebook/mbart-large-50", return_dict=True
) 
tokenizer = MBart50TokenizerFast.from_pretrained(
    "facebook/mbart-large-50", src_lang="de_DE", tgt_lang="de_DE"
)
model.to("cuda:0")
model.model.encoder.embed_tokens

Embedding(250054, 1024, padding_idx=1)

Complete the cues:

In [ ]:
def complete_cues(row):
    try:
        output_texts = []
        scores = []
        for cue, example in zip(row["cue"], row["example"]):
            input_text = cue + " <mask>"
            
            # print(tokenizer.tokenize(input_text))
            
            target_length = int(1.5 * len(example))  # allow double the length of the original sentence
            
            outputs = tokenizer(input_text, return_tensors="pt")
            outputs = model.generate(outputs["input_ids"].to("cuda:0"), max_length=target_length, num_return_sequences=1,
                                     num_beams=10, output_scores=True, return_dict_in_generate=True)
            output_text = tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)
            score = torch.exp(outputs.sequences_scores)
            output_texts += output_text
            
            scores.append(score)
        print(row["pseudoword"], str(output_texts), str([float(score) for score in scores]))
        
        return pd.DataFrame([str(output_texts), str([float(score) for score in scores])])
    except Exception as e:
        print(".", end="")
        return pd.DataFrame([row["pseudoword"], str(e), "[-1.0]"])

examples_reset = examples.reset_index()
pseudoword_output_scores = examples_reset[["example", "cue", "pseudoword"]].progress_apply(complete_cues, axis=1)
pseudoword_output_scores

  0%|          | 0/562 [00:00<?, ?it/s]

Und5 ['""Nicht herauskaufen"", ""nicht erpressen lassen"" wolle man sich. Und ""nicht erpressen lassen"".', 'Auch unter Schwarz-Gelb keiner wetten.', 'Er glaubt nicht an mehr Förderung, nicht an kleinere Klassen, nicht an individualisierteren Unterricht. Und er glaubt auch nicht an die Zukunft.', 'Vom Unfall kam er durch Zufall zum Einfall, nämlich zu den Grundlagen der Schwerkraft, ohne die nirgends auf der Welt ein Sack Reis umfällt. Und das ist der Grund, warum er so erfolgreich ist.', 'Nun allerdings sieht es nicht mehr wirklich nach Zukunft aus. Und das ist auch gut so.', 'Es ist ein Ton, auf den man sich in der Diskussion mit China einstellen muss, ohne sich allerdings daran gewöhnen zu dürfen. Und das ist auch gut so.', 'Der Landkreis Lüchow-Dannenberg ist eine der ärmsten Regionen Deutschlands', 'Sie will diese Höhle aus Blech nicht verlassen, die sie abschirmt vor der Welt da draußen. Und sie will sie nicht verlassen.', 'Damit wird wiederum nicht alles falsch, was danach gesch

In [ ]:
examples = pseudoword_output_scores[["pseudoword", "generated", "score"]]

examples

#### Anzahl neuer Sätze

In [ ]:
examples.to_csv(f"../../out/comapp/mbart/data_mbart_vanilla.tsv", sep="\t", decimal=",")
examples.to_excel(f"../../out/comapp/mbart/data_mbart_vanilla.xlsx")

In [ ]:
pseudoword_output_scores.to_csv("../../out/comapp/mbart/data_mbart_vanilla_complete.tsv", sep="\t", decimal=",")